In [1]:
import pandas as pd
import torch

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/splade-ready/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
full_collection = pd.read_table('data/toy_data/full_collection/raw.tsv', sep='\t', header=None)

In [32]:
len(full_collection)

8519

In [21]:
full_collection[1][0]

'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'

In [11]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

In [12]:
MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

text = list(full_collection[1])

Downloading: 100%|██████████| 1.27k/1.27k [00:00<00:00, 537kB/s]
Downloading: 100%|██████████| 780k/780k [00:00<00:00, 2.47MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 1.48MB/s]
Downloading: 100%|██████████| 1.29M/1.29M [00:00<00:00, 3.09MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 166kB/s]
Downloading: 100%|██████████| 1.84k/1.84k [00:00<00:00, 457kB/s]
Downloading: 100%|██████████| 476M/476M [00:24<00:00, 20.7MB/s] 


NameError: name 'full_collection' is not defined

In [13]:
class_mapping

{0: 'arts_&_culture',
 1: 'business_&_entrepreneurs',
 2: 'celebrity_&_pop_culture',
 3: 'diaries_&_daily_life',
 4: 'family',
 5: 'fashion_&_style',
 6: 'film_tv_&_video',
 7: 'fitness_&_health',
 8: 'food_&_dining',
 9: 'gaming',
 10: 'learning_&_educational',
 11: 'music',
 12: 'news_&_social_concern',
 13: 'other_hobbies',
 14: 'relationships',
 15: 'science_&_technology',
 16: 'sports',
 17: 'travel_&_adventure',
 18: 'youth_&_student_life'}

In [48]:
len(text[1])

311

In [67]:
#tokens = tokenizer(text[0:10], truncation=True, max_length=512, return_tensors='pt')
tokens = tokenizer(text[0:10], truncation=True, max_length=512, padding='max_length', return_tensors='pt')

In [68]:
tokens

{'input_ids': tensor([[    0,   133,  2621,  ...,     1,     1,     1],
        [    0,  1106,     5,  ...,     1,     1,     1],
        [    0,   133,   129,  ...,     1,     1,     1],
        ...,
        [    0,   133,  2136,  ...,     1,     1,     1],
        [    0, 33295, 17444,  ...,     1,     1,     1],
        [    0,  6179,  7840,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [69]:
output = model(**tokens)

scores = output[0][0].detach().numpy()
scores = expit(scores)
print(scores)
print(sum(scores))

[0.01264997 0.04049299 0.01736691 0.03581383 0.00802592 0.00493251
 0.01345491 0.01676077 0.00823048 0.0103578  0.2774843  0.01549087
 0.33673996 0.02477628 0.00906948 0.8390681  0.03405136 0.02357959
 0.06382543]
1.792171472683549


In [74]:
output[0].detach().numpy().shape

(10, 19)

In [77]:
!export PYTHONPATH=$PYTHONPATH:$(pwd)

In [4]:
from splade.datasets.datasets import CollectionDatasetPreLoad
d_collection = CollectionDatasetPreLoad(data_dir='data/toy_data/full_collection/', id_style="row_id")

Preloading dataset


8519it [00:00, 202645.56it/s]


In [5]:
passages = []
for i in range(len(d_collection)):
    passages.append(d_collection[i][1])

In [6]:
len(passages)

8519

In [15]:
MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokens = tokenizer(passages[:20], truncation=True, max_length=512, padding='max_length', return_tensors='pt')

output = model(**tokens)


In [8]:
tokens

{'input_ids': tensor([[    0,   133,  2621,  ...,     1,     1,     1],
        [    0,  1106,     5,  ...,     1,     1,     1],
        [    0,   133,   129,  ...,     1,     1,     1],
        ...,
        [    0,  9344, 30300,  ...,     1,     1,     1],
        [    0,   250,  3723,  ...,     1,     1,     1],
        [    0,  1779,    16,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [20]:
batch_1 = output[0].detach().numpy()
batch_2 = output[0].detach().numpy()

In [18]:
import h5py

In [24]:
np.vstack([batch_1, batch_2]).shape

(40, 19)

In [27]:
empty = np.empty(shape=(0, 19))

In [28]:
np.vstack([empty, batch_1]).shape

(20, 19)

In [1]:
a = [12,3,4]
a[0:2]

[12, 3]

In [7]:
batch_size = 10
arr = range(100)
for i in range(5):
    print(f"indexing batch {i}")
    start = i * batch_size
    end = (i +1) * batch_size
    print(start, end)
    print(list(arr[start:end]))
    print("-=-------------", len(list(arr[start:end])))


indexing batch 0
0 10
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-=------------- 10
indexing batch 1
10 20
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
-=------------- 10
indexing batch 2
20 30
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
-=------------- 10
indexing batch 3
30 40
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
-=------------- 10
indexing batch 4
40 50
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
-=------------- 10


In [2]:
#==========================================
# Reading the generated topics back

In [4]:
raw_classifications = pd.read_csv('full_collection_classifications.csv')

In [14]:
transformed = expit(raw_classifications.drop('Unnamed: 0', axis=1))

In [15]:
import h5py

In [16]:
h5f = h5py.File('full_collection_classifications.h5', 'w')
h5f.create_dataset('classifications', data=transformed)

<HDF5 dataset "classifications": shape (8519, 19), type "<f8">